<a href="https://colab.research.google.com/github/nazzang49/boost-camp-projects/blob/main/assignments/(P05)Word_Representation_Practice.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Practice Word Representation by Local and Distributed
- (URL) https://wikidocs.net/22650

In [ ]:
# !pip install konlpy

In [5]:
from konlpy.tag import Okt
import re  
okt=Okt()  

token=re.sub("(\.)","","정부가 발표하는 물가상승률과 소비자가 느끼는 물가상승률은 다르다.")  
token=okt.morphs(token)

word2index={}  
bow=[]  
for voca in token:  
    if voca not in word2index.keys():  
        word2index[voca]=len(word2index)  
        bow.insert(len(word2index)-1,1) # default count = 1
    else:
        index=word2index.get(voca)
        bow[index]=bow[index]+1
print(word2index)

{'정부': 0, '가': 1, '발표': 2, '하는': 3, '물가상승률': 4, '과': 5, '소비자': 6, '느끼는': 7, '은': 8, '다르다': 9}


In [6]:
bow

[1, 2, 1, 1, 2, 1, 1, 1, 1, 1]

## Movie Recommendation System through TF-IDF
- (URL) https://wikidocs.net/24603
- (Dataset) https://www.kaggle.com/rounakbanik/the-movies-dataset
- (Task) Recommend the most similar movies through overview which is summary

In [1]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel

In [5]:
data = pd.read_csv('movies_metadata.csv', low_memory=False)
data.head(5)

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,popularity,poster_path,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",21.946943,/rhIRbceoE9lR4veEXuwCC2wARtG.jpg,"[{'name': 'Pixar Animation Studios', 'id': 3}]","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0
1,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,17.015539,/vzmL6fP7aPKNKPRTFnZmiUfciyV.jpg,"[{'name': 'TriStar Pictures', 'id': 559}, {'na...","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0
2,False,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",NaN,15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,11.7129,/6ksm1sjKMFLbO7UY2i6G1ju9SML.jpg,"[{'name': 'Warner Bros.', 'id': 6194}, {'name'...","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-12-22,0.0,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.5,92.0
3,False,NaN,16000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",NaN,31357,tt0114885,en,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",3.859495,/16XOMpEaLWkrcPqSQqhTmeJuqQl.jpg,[{'name': 'Twentieth Century Fox Film Corporat...,"[{'iso_3166_1': 'US', 'name': 'United States o...",1995-12-22,81452156.0,127.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Friends are the people who let you be yourself...,Waiting to Exhale,False,6.1,34.0
4,False,"{'id': 96871, 'name': 'Father of the Bride Col...",0,"[{'id': 35, 'name': 'Comedy'}]",NaN,11862,tt0113041,en,Father of the Bride Part II,Just when George Banks has recovered from his ...,8.387519,/e64sOI48hQXyru7naBFyssKFxVd.jpg,"[{'name': 'Sandollar Productions', 'id': 5842}...","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-02-10,76578911.0,106.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,False,5.7,173.0


In [6]:
data = data.head(20000)

In [10]:
# null and NaN => replace to ''
print(data['overview'].isnull().sum())
print(data['overview'].isna().sum())
data['overview'] = data['overview'].fillna('')

135
135


In [11]:
# TF-IDF
tfidf = TfidfVectorizer(stop_words='english')
tfidf_matrix = tfidf.fit_transform(data['overview'])

# (20000, 47487) means that 47487 words are used in 20000 documents
print(tfidf_matrix.shape)

(20000, 47487)


In [12]:
# calculate cosine similarity among movies
cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)

In [13]:
# extract indices and movies => get index through title
indices = pd.Series(data.index, index=data['title']).drop_duplicates()
print(indices.head())

title
Toy Story                      0
Jumanji                        1
Grumpier Old Men               2
Waiting to Exhale              3
Father of the Bride Part II    4
dtype: int64


In [14]:
idx = indices['Father of the Bride Part II']
print(idx)

4


In [17]:
def get_recommendations(title, cosine_sim=cosine_sim):
    idx = indices[title]

    # get whole similarity between idx and others
    sim_scores = list(enumerate(cosine_sim[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    
    # 0 => criteria
    top_sim_scores = sim_scores[1:11]
    top_sim_indices = [i[0] for i in top_sim_scores]
    return data['title'].iloc[top_sim_indices]

In [19]:
get_recommendations('Toy Story')

15348               Toy Story 3
2997                Toy Story 2
10301    The 40 Year Old Virgin
8327                  The Champ
1071      Rebel Without a Cause
11399    For Your Consideration
1932                  Condorman
3057            Man on the Moon
485                      Malice
11606              Factory Girl
Name: title, dtype: object

In [20]:
get_recommendations('The Dark Knight')

18252                      The Dark Knight Rises
1328                              Batman Returns
15511                 Batman: Under the Red Hood
150                               Batman Forever
19792    Batman: The Dark Knight Returns, Part 1
585                                       Batman
3095                Batman: Mask of the Phantasm
18035                           Batman: Year One
9230          Batman Beyond: Return of the Joker
10122                              Batman Begins
Name: title, dtype: object

## (Optional) Other Similarity Methods
- Euclidean Distance
- Jaccard Similarity

In [21]:
import numpy as np

# euclidean => get actual distance between query doc and others
def dist(x,y):   
    return np.sqrt(np.sum((x-y)**2))

In [25]:
doc1 = np.array((2,3,0,1))
doc2 = np.array((1,2,3,1))
doc3 = np.array((2,1,2,2))
docQ = np.array((1,1,0,1))
print(doc1)

[2 3 0 1]


In [23]:
# the most similar doc is doc1
print(dist(doc1,docQ))
print(dist(doc2,docQ))
print(dist(doc3,docQ))

2.23606797749979
3.1622776601683795
2.449489742783178


In [ ]:
# jaccard => inter / union

In [26]:
doc1 = "apple banana everyone like likey watch card holder"
doc2 = "apple banana coupon passport love you"

tokenized_doc1 = doc1.split()
tokenized_doc2 = doc2.split()

print(tokenized_doc1)
print(tokenized_doc2)

['apple', 'banana', 'everyone', 'like', 'likey', 'watch', 'card', 'holder']
['apple', 'banana', 'coupon', 'passport', 'love', 'you']


In [27]:
# get union
union = set(tokenized_doc1).union(set(tokenized_doc2))
print(union)

{'apple', 'likey', 'everyone', 'watch', 'love', 'passport', 'banana', 'coupon', 'card', 'you', 'like', 'holder'}


In [28]:
# get inter
intersection = set(tokenized_doc1).intersection(set(tokenized_doc2))
print(intersection)

{'apple', 'banana'}


In [29]:
print(len(intersection)/len(union))

0.16666666666666666
